<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# SQLite In-Class Exercise

In [1]:
import sqlite3 as db
import pandas as pd
import numpy as np

In [2]:
# Connect to a database (or create one if it doesn't exist)
conn = db.connect('example.db')

# Create a 'cursor' for executing commands
c = conn.cursor()

# First check if the table already exists and if so we will delete it
c.execute("DROP TABLE IF EXISTS Students")

# Create a table named "Students" with 3 columns: "duke_id" (string), "name" (string), "grad_year" (integer).
c.execute("CREATE TABLE Students (duke_id INTEGER PRIMARY KEY, name TEXT, grad_year INTEGER)")

# Commands to add data to our table
c.execute("INSERT INTO Students VALUES ('121', 'Reifschneider', 2025)")
c.execute("INSERT INTO Students VALUES ('225', 'Egger', 2023)")
c.execute("INSERT INTO Students VALUES ('767', 'Lin', 2022)")
c.execute("INSERT INTO Students VALUES ('988', 'Saha', 2022)")

# Commit the changes (make them permanent in the datbase)
conn.commit()

# List of items to add
more_students = [('734', 'Fox', 2025),
                 ('878', 'Lenz', 2023),
                 ('267', 'Glass', 2023)]

# '?' question marks are placeholders for the columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?, ?)', more_students)
conn.commit()

# Create Classes table
c.execute('DROP TABLE IF EXISTS Classes')
c.execute('CREATE TABLE Classes (duke_id INTEGER, course TEXT, grade REAL, PRIMARY KEY("duke_id","course"))')

students = [('121','AIPI 510',3.7),
            ('121','AIPI 520',4.0),
            ('121','AIPI 530',3.3),
            ('225','AIPI 510',4.0),
            ('225','AIPI 520',3.3),
            ('767','MENG 570',3.0),
            ('767','AIPI 510',4.0),
            ('988','MENG 570',4.0),
            ('988','AIPI 510',3.7),
            ('734','AIPI 510',4.0),
            ('734','AIPI 520',4.0),
            ('878','AIPI 510',3.0),
            ('878','AIPI 520',4.0)]

c.executemany('INSERT INTO Classes VALUES (?,?,?)',students)
conn.commit()

# Displays the results of your code
c.execute('SELECT * FROM Classes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Classes:", results)

# Create dataframe of students' major
majors_dict={'duke_id':['225','734','767','878','121','267'],
             'major':['Biology','Finance','CS','AI','CS','Biology']}
majors = pd.DataFrame(majors_dict)

# Create table Majors from dataframe in example.db
conn = db.connect('example.db')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS Majors")
majors.to_sql(name='Majors',con=conn,index=False)

Your results: 13 
The entries of Classes: [(121, 'AIPI 510', 3.7), (121, 'AIPI 520', 4.0), (121, 'AIPI 530', 3.3), (225, 'AIPI 510', 4.0), (225, 'AIPI 520', 3.3), (767, 'MENG 570', 3.0), (767, 'AIPI 510', 4.0), (988, 'MENG 570', 4.0), (988, 'AIPI 510', 3.7), (734, 'AIPI 510', 4.0), (734, 'AIPI 520', 4.0), (878, 'AIPI 510', 3.0), (878, 'AIPI 520', 4.0)]


## Practice problems
### Question 1
Complete the below function to calculate the average GPA for students from each major.  The function should return a list of tuples containing the major and corresponding average gpa

In [3]:
def avg_gpa_by_major(db_name):  
    '''
    Calculates the average GPA for students from each major

    Inputs:
        db_name(str): name of the database

    Returns:
        results(list): list of tuples each containing the major (str) and the average GPA (float)
    '''
    ### BEGIN SOLUTION ###
    conn = db.connect('example.db')
    c = conn.cursor()
    query = '''
            SELECT Majors.major, AVG(Classes.grade) 
            FROM Majors 
            INNER JOIN Classes ON Majors.duke_id = Classes.duke_id
            GROUP BY Majors.major
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    return results
    ### END SOLUTION ###

In [4]:
# Run function
gpas = avg_gpa_by_major('example.db')
assert len(gpas) == 4
assert len(gpas[0]) == 2
gpas

[('AI', 3.5), ('Biology', 3.65), ('CS', 3.6), ('Finance', 4.0)]

### Question 2
Complete the function below which returns a **pandas dataframe** containing the Duke ID (as the index), name, graduation year, major (or None if not declared), and GPA of all students who have taken classes.

In [20]:
def all_students(db_name):
    '''
    Queries database to create a dataframe containing name, graduation, major and GPA of all students who have taken 1+ classes

    Inputs:
        db_name(str): name of database

    Returns:
        df(DataFrame): dataframe containing student names as index and their graduation year, major and GPA as columns
    '''
    ### BEGIN SOLUTION ###
    # Connect to database
    conn = db.connect('example.db')

    # Create a summary table of all students and their name, graduation year, major, GPA
    query = '''
            SELECT Students.duke_id, Students.name, Students.grad_year, Majors.major, AVG(Classes.grade) AS gpa
            FROM Students 
            INNER JOIN Classes ON Students.duke_id = Classes.duke_id
            LEFT JOIN Majors ON Students.duke_id = Majors.duke_id
            GROUP BY Students.duke_id
            '''
    # Create dataframe from SQL query
    df = pd.read_sql_query (query, conn)
    df.set_index(keys=['duke_id'],inplace=True)
    
    conn.close()
    return df
    ### END SOLUTION ###

In [21]:
# Run function
students_table = all_students('example.db')
assert students_table.shape == (6,4)
students_table

,name,grad_year,major,gpa
duke_id,,,,
121,Reifschneider,2025,CS,3.666667
225,Egger,2023,Biology,3.650000
734,Fox,2025,Finance,4.000000
767,Lin,2022,CS,3.500000
878,Lenz,2023,AI,3.500000
988,Saha,2022,None,3.850000


### Question 3
Complete the below function to create a pandas dataframe where the index values are the courses, the columns are the majors, and the cells show the number of students from each major in each course.  Your output should look like this:  
<img align="left" style="padding-top:10px;" src=Q3.png>

In [22]:
def majors_per_course(db_name):  
    '''
    Query the database to build a dataframe containing the number of students from each major in each course

    Inputs:
        db_name(str): name of the database

    Returns:
        df(DataFrame): dataframe with courses as index values and majors as the columns which contains the count of each major in each course
    '''
    ### BEGIN SOLUTION ###
    conn = db.connect('example.db')
    query = '''
            SELECT Classes.course, Majors.major, COUNT(*) AS count
            FROM Classes INNER JOIN Majors ON Classes.duke_id = Majors.duke_id
            GROUP BY Classes.course, Majors.major
            '''
    df = pd.read_sql_query (query, conn)
    conn.close()
    df =df.pivot(index='course',columns='major',values='count')
    df = df.fillna(0).astype(int)
    return df
    ### END SOLUTION ###

In [23]:
table = majors_per_course('example.db')
display(table)
assert table.shape == (4,4)

major,AI,Biology,CS,Finance
course,,,,
AIPI 510,1,1,2,1
AIPI 520,1,1,1,1
AIPI 530,0,0,1,0
MENG 570,0,0,1,0
